### <a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>Neural ProPhet - V3.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [1]:
# Importar las bibliotecas necesarias
import pandas as pd
from neuralprophet import NeuralProphet
import matplotlib.pyplot as plt
#
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')
productos_descripcion = pd.read_csv('data/tb_productos_descripcion.txt', delimiter='\t')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1600)

# Muestra las primeras filas del DataFrame
print(data_full.head())
data_full.info()
print(df_pid_validos.head())
print(productos_descripcion.head())
productos_descripcion.info()

#  Si tomo el archivo desde el buket de la maquina virtual del laboratorio
#
# Ruta del archivo ventas comprimido
# file_path = 'data/sell-in.txt.gz'

# Abre el archivo comprimido
#with gzip.open(file_path, 'rt') as f:
#    # Lee el archivo CSV usando pandas, especificando que el separador es un tabulador
#    data_full = pd.read_csv(f, sep='\t')



   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_

In [3]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [4]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

# Ver como esta el data frame
data.info()
data

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452
...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230
2945814,201912,10092,20853,0,1,0.00669,0.00669
2945815,201912,10006,20853,0,7,0.02898,0.02898
2945816,201912,10018,20853,0,4,0.01561,0.01561


In [5]:
# Cargar el DataFrame de stocks
stocks = pd.read_csv('data/tb_stocks.csv', delimiter='\t')
stocks = stocks.copy()

In [6]:
stocks.info()
stocks

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13691 entries, 0 to 13690
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   periodo      13691 non-null  int64  
 1   product_id   13691 non-null  int64  
 2   stock_final  13691 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 321.0 KB


,periodo,product_id,stock_final
0,201810,20524,1.61267
1,201810,20311,2.93657
2,201810,20654,6.83269
3,201810,21005,1.01338
4,201810,20974,0.34595
...,...,...,...
13686,201912,20453,1.43741
13687,201912,21026,7.26817
13688,201912,21054,0.50833
13689,201912,20981,2.18491


In [7]:
import pandas as pd

# Asegurar que ambos DataFrames tienen el mismo formato de periodo
data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')
stocks['periodo'] = pd.to_datetime(stocks['periodo'], format='%Y%m')

# Verificar si la columna 'stock_final' está presente en el DataFrame 'stocks'
if 'stock_final' not in stocks.columns:
    raise KeyError("'stock_final' no está presente en el DataFrame 'stocks'")

# Usar .loc para agregar las columnas de año y mes
data.loc[:, 'year'] = data['periodo'].dt.year
data.loc[:, 'month'] = data['periodo'].dt.month
stocks.loc[:, 'year'] = stocks['periodo'].dt.year
stocks.loc[:, 'month'] = stocks['periodo'].dt.month

# Verificar si las columnas year y month están presentes
print(data.columns)
print(stocks.columns)

# Eliminar las columnas duplicadas de uno de los DataFrames antes de la combinación
stocks = stocks.drop(columns=['year', 'month'])

# Combinar los DataFrames en base a periodo y product_id
data = data.merge(stocks, on=['periodo', 'product_id'], how='left')

# Verificar si la combinación se realizó correctamente
print(data.columns)

# Verificar si la columna 'stock_final' está presente después de la combinación
if 'stock_final' not in data.columns:
    raise KeyError("'stock_final' no está presente en el DataFrame 'data' después de la combinación")

# Calcular la nueva variable: ratio entre tn y stock_final
data['tn_stock_ratio'] = data['tn'] / (data['stock_final'] + 1e-9)  # Añadir un pequeño valor para evitar la división por cero

# Agregar ventas por producto, cliente y período
data_agg = data.groupby(['product_id', 'customer_id', 'year', 'month']).agg({
    'tn': 'sum',
    'plan_precios_cuidados': 'max',  # Asumimos que si algún registro tiene 1, el producto estaba en el plan
    'cust_request_qty': 'sum',
    'tn_stock_ratio': 'mean'  # Tomamos el promedio del ratio
}).reset_index()

# Filtrar para los productos seleccionados
def filter_products(data, product_id_min, product_id_max):
    return data[(data['product_id'] >= product_id_min) & (data['product_id'] <= product_id_max)]

# Filtrar productos (ajusta estos valores según tus necesidades)
product_id_min = 20000
product_id_max = 20005   #21276
filtered_data = filter_products(data_agg, product_id_min, product_id_max)

# Crear variables dummy para product_id (opcional)
# product_dummies = pd.get_dummies(filtered_data['product_id'], prefix='prod')
# filtered_data = pd.concat([filtered_data, product_dummies], axis=1)

# Recalcular características históricas
filtered_data = filtered_data.sort_values(['product_id', 'customer_id', 'year', 'month'])
filtered_data['hist_sales'] = filtered_data.groupby(['product_id', 'customer_id'])['tn'].shift(1).fillna(0)
filtered_data['cum_sales'] = filtered_data.groupby(['product_id', 'customer_id'])['tn'].cumsum() - filtered_data['tn']

# Resultado final
print(filtered_data)


Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn', 'year', 'month'], dtype='object')
Index(['periodo', 'product_id', 'stock_final', 'year', 'month'], dtype='object')
Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn', 'year', 'month', 'stock_final'], dtype='object')
       product_id  customer_id  year  month         tn  plan_precios_cuidados  cust_request_qty  tn_stock_ratio  hist_sales  cum_sales
0           20001        10001  2017      1   99.43861                      0                11             NaN     0.00000    0.00000
1           20001        10001  2017      2  198.84365                      0                23             NaN    99.43861   99.43861
2           20001        10001  2017      3   92.46537                      0                33             NaN   198.84365  298.28226
3           20001        10001  2017      4   13.29728        

In [8]:
filtered_data

,product_id,customer_id,year,month,tn,plan_precios_cuidados,cust_request_qty,tn_stock_ratio,hist_sales,cum_sales
0,20001,10001,2017,1,99.43861,0,11,NaN,0.00000,0.00000
1,20001,10001,2017,2,198.84365,0,23,NaN,99.43861,99.43861
2,20001,10001,2017,3,92.46537,0,33,NaN,198.84365,298.28226
3,20001,10001,2017,4,13.29728,0,8,NaN,92.46537,390.74763
4,20001,10001,2017,5,101.00563,0,15,NaN,13.29728,404.04491
...,...,...,...,...,...,...,...,...,...,...
32010,20005,10611,2017,10,0.08590,0,1,NaN,0.08590,0.42951
32011,20005,10613,2017,3,0.08590,0,1,NaN,0.00000,0.00000
32012,20005,10615,2017,1,0.04295,0,1,NaN,0.00000,0.00000
32013,20005,10615,2017,5,0.12886,0,2,NaN,0.04295,0.04295


In [11]:
# Preprocesar y combinar los datos
filtered_data['date'] = pd.to_datetime(filtered_data[['year', 'month']].assign(day=1))
combined_data = pd.merge(filtered_data, productos_descripcion, on='product_id', how='left')

# Seleccionar las columnas relevantes y renombrar para NeuralProphet
combined_data = combined_data[['date', 'product_id', 'tn', 'cat1', 'cat2', 'cat3', 'customer_id', 'sku_size', 'hist_sales', 'tn_stock_ratio']]
combined_data.rename(columns={'date': 'ds', 'tn': 'y'}, inplace=True)

# Rellenar NaN con ceros (o cualquier valor apropiado)
combined_data['sku_size'].fillna(0, inplace=True)
combined_data['hist_sales'].fillna(0, inplace=True)
combined_data['tn_stock_ratio'].fillna(0, inplace=True)

# Realizar one-hot encoding de customer_id
combined_data = pd.get_dummies(combined_data, columns=['customer_id'], drop_first=True)

# Agrupar los datos por categorías y product_id
grouped_data = combined_data.groupby(['cat1', 'cat2', 'cat3', 'product_id'])

predictions = []

for (cat1, cat2, cat3, product_id), group in grouped_data:
    try:
        print(f"Training model for category: {cat1}, {cat2}, {cat3}, product_id: {product_id}")
        
        # Comprobar que hay suficientes datos para entrenar
        if len(group) < 6:  # Asegurarse de que hay al menos 6 meses de datos para las lags
            print(f"Not enough data to train model for product_id: {product_id}")
            continue
        
        # Inicializar un nuevo modelo para cada grupo
        model = NeuralProphet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        
        # Añadir estacionalidad trimestral
        model.add_seasonality(name='quarterly', period=91.25, fourier_order=5)
        
        # Añadir variables autoregresivas (lags) para 6 meses
        regressors = ['sku_size', 'hist_sales', 'tn_stock_ratio']
        for regressor in regressors:
            model = model.add_lagged_regressor(names=regressor, n_lags=6)
        
        # Añadir customer_id como regresores futuros (one-hot encoded)
        customer_id_columns = [col for col in group.columns if col.startswith('customer_id_')]
        for customer_col in customer_id_columns:
            model.add_future_regressor(name=customer_col)
        
        # Entrenar el modelo
        group = group[['ds', 'y'] + regressors + customer_id_columns]
        model.fit(group, freq='M')
        
        future = model.make_future_dataframe(df=group, periods=2, n_historic_predictions=True)
        
        # Agregar valores futuros para los regresores
        for regressor in regressors + customer_id_columns:
            future[regressor] = group[regressor].values[-1]
        
        forecast = model.predict(future)
        
        # Filtrar la predicción para el mes de febrero de 2020
        forecast_feb_2020 = forecast[forecast['ds'] == '2020-02-01']
        if not forecast_feb_2020.empty:
            tn_pred = forecast_feb_2020['yhat1'].values[0]
            tn_pred = max(tn_pred, 0)  # Ajustar para que no haya valores negativos
            predictions.append({
                'product_id': product_id,
                'cat1': cat1,
                'cat2': cat2,
                'cat3': cat3,
                'customer_id': group[customer_id_columns].idxmax(axis=1).values[-1],  # Obtener el último customer_id
                'sku_size': group['sku_size'].values[-1],
                'tn': tn_pred
            })
    except Exception as e:
        print(f"Error processing product_id: {product_id}. Error: {e}")
        continue

# Crear DataFrame de predicciones y guardar en CSV
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('NeuralProphet.v3.0.csv', index=False)

print("Predicciones guardadas en NeuralProphet.v3.0.csv")

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.


Training model for category: FOODS, ADEREZOS, Mayonesa, product_id: 20003
Error processing product_id: 20003. Error: Column ds has duplicate values. Please remove duplicates.
Training model for category: FOODS, ADEREZOS, Mayonesa, product_id: 20004
Error processing product_id: 20004. Error: Column ds has duplicate values. Please remove duplicates.
Training model for category: FOODS, ADEREZOS, Mayonesa, product_id: 20005
Error processing product_id: 20005. Error: Column ds has duplicate values. Please remove duplicates.
Training model for category: HC, ROPA LAVADO, Liquido, product_id: 20001
Error processing product_id: 20001. Error: Column ds has duplicate values. Please remove duplicates.
Training model for category: HC, ROPA LAVADO, Liquido, product_id: 20002
Error processing product_id: 20002. Error: Column ds has duplicate values. Please remove duplicates.
Predicciones guardadas en NeuralProphet.v3.0.csv


<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [ ]:
# Inicialización del modelo
model = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    growth='linear',
    n_changepoints=15,
    changepoint_range=0.9,
    learning_rate=0.01,
    epochs=100,
    batch_size=32,
    normalize='auto'
)

# Añadir estacionalidad trimestral
model.add_seasonality(name='quarterly', period=91.25, fourier_order=5)

# Añadir regresores adicionales
model.add_regressor(name='hist_sales')
model.add_regressor(name='sku_size')
model.add_regressor(name='tn_stock_ratio')

# Ajustar el modelo con los datos
model.fit(df)

# Hacer predicciones
future = model.make_future_dataframe(df, periods=12, n_historic_predictions=True)
forecast = model.predict(future)

# Visualizar resultados
model.plot(forecast)